In [23]:
!pip freeze | grep scikit-learn

'grep' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [24]:
!python -V

Python 3.9.21


In [25]:
import pickle
import pandas as pd

In [26]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

c:\Users\gvdp5\anaconda3\envs\mlops_course\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\gvdp5\anaconda3\envs\mlops_course\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [27]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [28]:
month = '05'
year = '2023'

df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month}.parquet')

In [29]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [30]:
y_pred.std()

6.263976619625352

In [31]:
y_pred.mean()

14.242595513316317

In [32]:
df['ride_id'] = f'{year}/{month}_' + df.index.astype('str')

In [33]:
output_file = f'yellow_tripdata_{year}-{month}_predictions.parquet'
df_result = pd.DataFrame({
    'ride_id': df['ride_id'],
    'prediction': y_pred
})

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)